- **TODO:** train, test, main
- **TODO:** plot_accuracies_losses.py

In [27]:
import os
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torch.utils.data.sampler import SubsetRandomSampler
from pathlib import Path
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

In [18]:
# main.py

# root_dir = "datasets/office/amazon/images"
# path = "datasets/office/amazon"
# name_dataset = "office"

data_domain = "amazon"
img_size = 224
#
# Pytorch ImageFolder fits our dataset structure
# dataset = datasets.ImageFolder(root=root_dir, transform=data_transform)
image_data_folder = datasets.ImageFolder(root= "datasets/office/%s/images" % data_domain)
print("total no. of images in amazon dataset:",len(image_data_folder))

data_transforms = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


total no. of images in amazon dataset: 2817


In [19]:
# dataloader.py
class OfficeAmazonDataset(Dataset):
    """Class to create an iterable dataset 
    of images and corresponding labels """
        
    def __init__(self, image_folder_dataset, transform=None):
        super(OfficeAmazonDataset, self).__init__()
        self.image_folder_dataset = image_folder_dataset
        self.transform = transform

    def __len__(self):
        return len(self.image_folder_dataset.imgs)
    
    def __getitem__(self, idx):
        # read image, class from folder_dataset given index
        img, img_label = image_folder_dataset[idx][0], image_folder_dataset[idx][1]
        
        # apply transformations (it already returns them as torch tensors)
        if self.transform is not None:
            self.transform(img)
        
        img_label_pair = {"image": img,
                         "class": img_label}
        
        return img_label_pair

def get_dataloader(dataset, batch_size):
        
    def get_subset(indices, start, end):
        return indices[start:start + end]
    
    # Split train/val data ratios
    TRAIN_RATIO, VALIDATION_RATIO = 0.7, 0.3
    train_set_size = int(len(dataset) * TRAIN_RATIO)
    validation_set_size = int(len(dataset) * VALIDATION_RATIO)

    # Generate random indices for train and val sets
    indices = torch.randperm(len(dataset))
    train_indices = get_subset(indices, 0, train_set_size)
    validation_indices = get_subset(indices, train_set_size, validation_set_size)
    # test_indices = get_subset(indices, train_count + validation_count, len(dataset))

    # Create sampler objects 
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(validation_indices)

    # Create data loaders for data
    train_loader = DataLoader(dataset, batch_size=batch_size, 
                              sampler=train_sampler, num_workers=4)

    val_loader = DataLoader(dataset, batch_size=batch_size, 
                            sampler=val_sampler, num_workers=4)
    
    return train_loader, val_loader

In [20]:
amazon_dataset = OfficeAmazonDataset(image_folder_dataset=image_data_folder, transform=data_transforms)
train_loader, val_loader = get_dataloader(amazon_dataset, batch_size=4)

### mean and std of dataset

In [10]:
import torch
from torchvision import transforms, datasets
import numpy as np


def compute_mean_std(path_dataset):
    """
    Compute mean and standard deviation of an image dataset.
    Acknowledgment : http://forums.fast.ai/t/image-normalization-in-pytorch/7534
    """
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor()
    ])

    dataset = datasets.ImageFolder(root=path_dataset,
                                   transform=transform)
    # Choose a large batch size to better approximate. Optimally load the dataset entirely on memory.
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=4096, shuffle=False, num_workers=4)

    pop_mean = []
    pop_std = []

    for i, data in enumerate(data_loader, 0):
        # shape (batch_size, 3, height, width)
        numpy_image = data[0].numpy()

        # shape (3,) -> 3 channels
        batch_mean = np.mean(numpy_image, axis=(0, 2, 3))
        batch_std = np.std(numpy_image, axis=(0, 2, 3))

        pop_mean.append(batch_mean)
        pop_std.append(batch_std)

    # shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
    pop_mean = np.array(pop_mean).mean(axis=0)
    pop_std = np.array(pop_std).mean(axis=0)

    values = {
        'mean': pop_mean,
        'std': pop_std
    }

    return values

In [ ]:
# retrieved from: https://forums.fast.ai/t/image-normalization-in-pytorch/7534/7
# this cell takes too much memory and time to compute
data_domain = "amazon"
path_dataset = "datasets/office/%s/images" % data_domain

transform = transforms.Compose([
        transforms.Resize((224, 224)), # original image size 300x300 pixels
        transforms.ToTensor()])

dataset = datasets.ImageFolder(root=path_dataset,
                               transform=transform)

# set large batch size to get good approximate of mean, std of full dataset
data_loader = DataLoader(dataset, batch_size=2048, shuffle=False, num_workers=0) # 4096

mean = []
std = []

# for i, data in enumerate(data_loader, 0):
    # shape is (batch_size, channels, height, width)
    npy_image = data[0].numpy()
    
    # compute mean, std per batch shape (3,) three channels
    batch_mean = np.mean(npy_image, axis=(0,2,3))
    batch_std = np.std(npy_image, axis=(0,2,3))
    
    mean.append(batch_mean)
    std.append(batch_std)
    
# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
mean = np.array(mean).mean(axis=0) # average over batch averages
std = np.arry(std).mean(axis=0) # average over batch stds

values = {
    "mean": mean,
    "std": std
}

### testing dataloader for office dataset

In [14]:
from dataloader import get_office_dataloader
amazon_dataloader = get_office_dataloader("amazon", 2048)
amazon_dataloader.batch_size

2048

## Putting all together

In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import numpy as np
import os
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from utils import get_mean_std_dataset


"""
Created on Saturday Feb 22 2020

@authors: Alan Preciado, Santosh Muthireddy
"""
class OfficeAmazonDataset(Dataset):
    """Class to create an iterable dataset
    of images and corresponding labels """

    def __init__(self, image_folder_dataset, transform=None):
        super(OfficeAmazonDataset, self).__init__()
        self.image_folder_dataset = image_folder_dataset
        self.transform = transform

    def __len__(self):
        return len(self.image_folder_dataset.imgs)

    def __getitem__(self, idx):
        # read image, class from folder_dataset given index
        img, img_label = image_folder_dataset[idx][0], image_folder_dataset[idx][1]

        # apply transformations (it already returns them as torch tensors)
        if self.transform is not None:
            self.transform(img)

        img_label_pair = {"image": img,
                         "class": img_label}

        return img_label_pair


def get_dataloader(dataset, batch_size, train_ratio=0.7):
    """
    Splits a dataset into train and test.
    Returns train_loader and test_loader.
    """

    def get_subset(indices, start, end):
        return indices[start:start+end]

    # Split train/val data ratios
    TRAIN_RATIO, VALIDATION_RATIO = train_ratio, 1-train_ratio
    train_set_size = int(len(dataset) * TRAIN_RATIO)
    validation_set_size = int(len(dataset) * VALIDATION_RATIO)

    # Generate random indices for train and val sets
    indices = torch.randperm(len(dataset))
    train_indices = get_subset(indices, 0, train_set_size)
    validation_indices = get_subset(indices, train_set_size, validation_set_size)
    # test_indices = get_subset(indices,train_count+validation_count,len(dataset))

    # Create sampler objects
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(validation_indices)

    # Create data loaders
    train_loader = DataLoader(dataset, batch_size=batch_size,
                              sampler=train_sampler, num_workers=4)

    val_loader = DataLoader(dataset, batch_size=batch_size,
                            sampler=val_sampler, num_workers=4)

    return train_loader, val_loader


def get_office_dataloader(name_dataset, batch_size, train=True):
    """
    Creates dataloader for the datasets in office datasetself.
    Makes use of get_mean_std_dataset to compute mean and std along the
    color channels for each dataset in office.
    """
    # Ideally compute mean and std with get_mean_std_dataset.py
    # Values retrieved from:
    # https://github.com/DenisDsh/PyTorch-Deep-CORAL/blob/master/data_loader.py

    root_dir = "datasets/office/%s/images" % name_dataset

    __datasets__ = ["amazon", "dslr", "webcam"]

    if name_dataset not in __datasets__:
        raise ValueError("must introduce one of the three datasets in office")

    mean_std = {
        "amazon":{
            "mean":[0.7923, 0.7862, 0.7841],
            "std":[0.3149, 0.3174, 0.3193]
        },
        "dslr":{
            "mean":[0.4708, 0.4486, 0.4063],
            "std":[0.2039, 0.1920, 0.1996]
        },
        "webcam":{
            "mean":[0.6119, 0.6187, 0.6173],
            "std":[0.2506, 0.2555, 0.2577]
        }
    }

    data_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            # transforms.RandomSizedCrop(224),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean_std[name_dataset]["mean"],
                                 std=mean_std[name_dataset]["std"])
        ])

    dataset = datasets.ImageFolder(root=root_dir,
                                   transform=data_transforms)

    # note on shuffling in data loader:
    # https://stackoverflow.com/questions/54354465/impact-of-using-data-shuffling-in-pytorch-dataloader
    dataset_loader = DataLoader(dataset,
                                batch_size=batch_size,
                                shuffle=train,
                                num_workers=4)

    return dataset_loader

In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch


"""
Created on Saturday Feb 25 2020

@authors: Alan Preciado, Santosh Muthireddy
"""
def CORAL_loss(source, target):
    """
    From the paper, the vectors that compose Ds and Dt are D-dimensional vectors.
    :param source: torch tensor: source data (Ds) with dimensions DxNs
    :param target: torch tensor: target data (Dt) with dimensons DxNt
    """

    d = source.size(1) # d-dimensional vectors (same for source, target)

    source_covariance = compute_covariance(source)
    target_covariance = compute_covariance(target)

    # take Frobenius norm (https://pytorch.org/docs/stable/torch.html)
    loss = torch.norm(torch.mul((source_covariance-target_covariance),(source_covariance-target_covariance)), p="fro")
    # loss = torch.norm(torch.mm((source_covariance-target_covariance),(source_covariance-target_covariance)), p="fro")

    loss = loss / (4*d*d)

    return loss


def compute_covariance(data):
    """
    Compute covariance matrix for given dataset as shown in paper.
    Equations 2 and 3.
    :param data: torch tensor: input source/target data
    """

    # data dimensions: nxd (this for Ns or Nt)
    n = data.size(0)

    # proper matrix multiplication for right side of equation (2)
    ones_vector = torch.ones(n).resize(1, n) 	# 1xN dimensional vector (transposed)
    one_onto_D = torch.mm(ones_vector, data)
    mult_right_terms = torch.mm(one_onto_D.t(), one_onto_D)
    mult_right_terms = torch.div(mult_right_terms, n) # element-wise divison

    # matrix multiplication for left side of equation (2)
    mult_left_terms = torch.mm(data.t(), data)

    covariance_matrix = 1/(n-1) * torch.add(mult_left_terms, -1*(mult_right_terms))

    return covariance_matrix

In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from skimage import io
import matplotlib.pyplot as plt
import os
import torch
from torchvision import transforms, datasets
# from torch.utils import model_zoo
import numpy as np
try: # import pytorch data getters
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url


"""
Created on Saturday Feb 22 2020

@authors: Alan Preciado, Santosh Muthireddy
"""
def load_pretrained_AlexNet(model, progress=True):
# def alexnet(pretrained=False, progress=True, **kwargs):
    """
    AlexNet model architecture from the paper
    pretrained (bool): If True, returns a model pre-trained on ImageNet
    progress (bool): If True, displays a progress bar of the download to stderr
    """

    __all_ = ["AlexNet", "alexnet", "Alexnet"]

    model_url = {
        'alexnet':'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
    }

    state_dict = load_state_dict_from_url(model_url['alexnet'], progress=progress)
    model_dict = model.state_dict() # check this one

    # filter out unmatching dictionary
    # reference: https://github.com/SSARCandy/DeepCORAL/blob/master/main.py
    state_dict = {k: v for k, v in state_dict.items() if k in model_dict}

    model_dict.update(state_dict)
    model.load_state_dict(state_dict)


def show_image(dataset, domain, image_class, image_name):
    """
    Plot images from given domain, class
    """
    image_file = io.imread(os.path.join("data", dataset, domain, "images", image_class, image_name))
    plt.imshow(image_file)
    plt.pause(0.001)
    plt.figure()


def accuracy(prediction, target):
    pass


def save_model(model, path):
    torch.save(model.state_dict(), path)
    print("checkpoint saved in {}".format(path))


def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print("checkpoint loaded from {}".format(path))


def get_mean_std_dataset(root_dir):
    """
    Function to compute mean and std of image dataset.
    Move batch_size param according to memory resources.
    retrieved from: https://forums.fast.ai/t/image-normalization-in-pytorch/7534/7
    """

    # data_domain = "amazon"
    # path_dataset = "datasets/office/%s/images" % data_domain

    transform = transforms.Compose([
            transforms.Resize((224, 224)), # original image size 300x300 pixels
            transforms.ToTensor()])

    dataset = datasets.ImageFolder(root=root_dir,
                                   transform=transform)

    # set large batch size to get good approximate of mean, std of full dataset
    # batch_size: 4096, 2048
    data_loader = DataLoader(dataset, batch_size=2048,
                            shuffle=False, num_workers=0)

    mean = []
    std = []

    for i, data in enumerate(data_loader, 0):
        # shape is (batch_size, channels, height, width)
        npy_image = data[0].numpy()

        # compute mean, std per batch shape (3,) three channels
        batch_mean = np.mean(npy_image, axis=(0,2,3))
        batch_std = np.std(npy_image, axis=(0,2,3))

        mean.append(batch_mean)
        std.append(batch_std)

    # shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
    mean = np.array(mean).mean(axis=0) # average over batch averages
    std = np.arry(std).mean(axis=0) # average over batch stds

    values = {
        "mean": mean,
        "std": std
    }

    return values

In [24]:
# main
from __future__ import division
import argparse
import torch
from torch.autograd import Variable

# import models --> DeepCoral, AlexNet
# import utils --> load_alex_net, save, load
# from data_loader import get_office_dataloader

CUDA = True if torch.cuda.is_available() else False
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9
BATCH_SIZE = [200, 56] # source, target
EPOCHS = 2

# create dataloaders (amazon source vs webcam target)
source_loader = get_office_dataloader(name_dataset="amazon", batch_size=BATCH_SIZE[0])
target_loader = get_office_dataloader(name_dataset="webcam", batch_size=BATCH_SIZE[1])

def train(model, optimizer, epoch, _lambda):
    """
    This method fits the network params one epoch at a time.
    Implementation based on: https://github.com/SSARCandy/DeepCORAL/blob/master/main.py
    """
    pass

In [ ]:
list(enumerate(source_loader))